In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 6.44 ms (started: 2021-08-13 01:29:26 +08:00)


In [ ]:
%%bash

# 增加更新
git add *.ipynb

git remote -v

git commit -m '更新 #1 Aug 12, 2021'

git push origin master

In [2]:
#设置使用的gpu
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 1.42 s (started: 2021-08-13 01:29:26 +08:00)


# 安装

In [ ]:
!pip install keras-tuner -q

# 介绍

以下是如何使用随机搜索为单层密集神经网络执行超参数调整。 首先，我们需要准备数据集——让我们以 MNIST 数据集为例。

In [3]:
from tensorflow import keras
import numpy as np

(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

time: 636 ms (started: 2021-08-13 01:29:36 +08:00)


In [3]:
x_train.shape

(50000, 28, 28, 1)

time: 14 ms (started: 2021-08-13 01:10:08 +08:00)


# 准备模型构建函数

然后，我们定义一个模型构建函数。 它接受一个参数 hp，您可以从中采样超参数，例如`hp.Int('units', min_value=32, max_value=512, step=32)` （某个范围内的整数）。

此函数返回一个编译模型。

In [4]:
from tensorflow.keras import layers
from keras_tuner import RandomSearch


def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            activation="relu",
        )
    )
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

time: 386 ms (started: 2021-08-13 01:29:39 +08:00)


# 开始搜索
接下来，让我们实例化一个调谐器。 您应该指定模型构建函数、要优化的目标的名称（对于内置指标自动推断是最小化还是最大化）、要测试的试验总数 (`max_trials`) 以及应该使用的模型数 构建并适合每次试验 (`executions_per_trial`)。

我们使用 `overwrite` 参数来控制是覆盖同一目录中的先前结果还是恢复先前的搜索。 在这里，我们设置 `overwrite=True` 以开始新的搜索并忽略任何先前的结果。

可用的调谐器有 `RandomSearch`、`BayesianOptimization` 和 `Hyperband`。

> **注意**：每次试验多次执行的目的是减少结果差异，从而能够更准确地评估模型的性能。 如果您想更快地获得结果，您可以设置 executions_per_trial=1（每个模型配置的单轮训练）。

In [5]:
tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

time: 1.42 s (started: 2021-08-13 01:29:41 +08:00)


您可以打印搜索空间的摘要：

In [7]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
time: 1.43 ms (started: 2021-08-13 01:29:45 +08:00)


然后，开始搜索最佳超参数配置。 对搜索的调用与 model.fit() 具有相同的签名。

In [8]:
tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Trial 3 Complete [00h 01m 40s]
val_accuracy: 0.9417499899864197

Best val_accuracy So Far: 0.9543499946594238
Total elapsed time: 00h 05m 01s
INFO:tensorflow:Oracle triggered exit
time: 5min 1s (started: 2021-08-13 01:30:10 +08:00)


以下是搜索中发生的事情：通过调用模型构建函数迭代地构建模型，该函数填充由 hp 对象跟踪的超参数空间（搜索空间）。 调谐器逐步探索空间，记录每个配置的指标。

# 查询结果

搜索结束后，您可以检索最佳模型：

In [9]:
models = tuner.get_best_models(num_models=2)

time: 297 ms (started: 2021-08-13 01:37:44 +08:00)


或打印结果摘要：

In [10]:
tuner.results_summary()

Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
units: 320
learning_rate: 0.01
Score: 0.9543499946594238
Trial summary
Hyperparameters:
units: 32
learning_rate: 0.01
Score: 0.9505999982357025
Trial summary
Hyperparameters:
units: 288
learning_rate: 0.0001
Score: 0.9417499899864197
time: 2.43 ms (started: 2021-08-13 01:38:05 +08:00)


您还可以在文件夹 my_dir/helloworld 中找到详细的日志、检查点等，即目录/项目名称。

# 搜索空间可能包含条件超参数

下面，我们有一个 for 循环，用于创建可调数量的层，这些层本身涉及一个可调单位参数。

这可以推到任何级别的参数相互依赖性，包括递归。

请注意，所有参数名称都应该是唯一的（这里，在 i 上的循环中，我们将内部参数命名为“units_”+ str(i)）。

In [18]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    for i in range(hp.Int("num_layers", 2, 20)):
        model.add(
            layers.Dense(
                units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
                activation="relu",
            )
        )
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

time: 1.58 ms (started: 2021-08-13 01:44:18 +08:00)


# 使用 HyperModel 子类而不是模型构建函数

这使得共享和重用超模型变得容易。

HyperModel 子类只需要实现一个 build(self, hp) 方法。

In [19]:
from keras_tuner import HyperModel


class MyHyperModel(HyperModel):
    def __init__(self, classes):
        self.classes = classes

    def build(self, hp):
        model = keras.Sequential()
        model.add(layers.Flatten())
        model.add(
            layers.Dense(
                units=hp.Int("units", min_value=32, max_value=512, step=32),
                activation="relu",
            )
        )
        model.add(layers.Dense(self.classes, activation="softmax"))
        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
            ),
            loss="categorical_crossentropy",
            metrics=["accuracy"],
        )
        return model

time: 2.36 ms (started: 2021-08-13 01:47:22 +08:00)


In [20]:
hypermodel = MyHyperModel(classes=10)

tuner = RandomSearch(
    hypermodel,
    objective="val_accuracy",
    max_trials=3,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Trial 3 Complete [00h 00m 49s]
val_accuracy: 0.9469000101089478

Best val_accuracy So Far: 0.975600004196167
Total elapsed time: 00h 02m 29s
INFO:tensorflow:Oracle triggered exit
time: 2min 30s (started: 2021-08-13 01:48:00 +08:00)


# 预制的可调应用程序：HyperResNet 和 HyperXception

这些是用于计算机视觉的即用型超模型。

它们使用 loss="categorical_crossentropy" 和 metrics=["accuracy"] 进行预编译。

In [21]:
from keras_tuner.applications import HyperResNet

hypermodel = HyperResNet(input_shape=(28, 28, 1), classes=10)

tuner = RandomSearch(
    hypermodel,
    objective="val_accuracy",
    max_trials=3,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

tuner.search(
    x_train[:100], y_train[:100], epochs=1, validation_data=(x_val[:100], y_val[:100])
)

Trial 3 Complete [00h 00m 14s]
val_accuracy: 0.10000000149011612

Best val_accuracy So Far: 0.10999999940395355
Total elapsed time: 00h 01m 45s
INFO:tensorflow:Oracle triggered exit
time: 1min 46s (started: 2021-08-13 01:51:39 +08:00)


# 可以轻松地将搜索空间限制为几个参数

如果您有一个现有的超模型，并且只想搜索几个参数（例如学习率），则可以通过将超参数参数传递给调谐器构造函数以及 tune_new_entries=False 来指定那些参数 你没有在超参数中列出不应该被调整。 对于这些参数，使用默认值。

In [22]:
from keras_tuner import HyperParameters
from keras_tuner.applications import HyperXception

hypermodel = HyperXception(input_shape=(28, 28, 1), classes=10)

hp = HyperParameters()

# This will override the `learning_rate` parameter with your
# own selection of choices
hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])

tuner = RandomSearch(
    hypermodel,
    hyperparameters=hp,
    # `tune_new_entries=False` prevents unlisted parameters from being tuned
    tune_new_entries=False,
    objective="val_accuracy",
    max_trials=3,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

tuner.search(
    x_train[:100], y_train[:100], epochs=1, validation_data=(x_val[:100], y_val[:100])
)

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.09000000357627869

Best val_accuracy So Far: 0.11999999731779099
Total elapsed time: 00h 00m 09s
INFO:tensorflow:Oracle triggered exit
time: 10.7 s (started: 2021-08-13 01:54:58 +08:00)


# 关于参数默认值

每当您在模型构建函数或超模型的构建方法中注册超参数时，您都可以指定默认值：

In [23]:
hp.Int("units", min_value=32, max_value=512, step=32, default=128)

128

time: 3.94 ms (started: 2021-08-13 01:56:04 +08:00)


如果你不这样做，超参数总是有一个默认的默认值（对于 Int，它等于 min_value）。

# 固定超模型中的值

如果你想做相反的事情——调整超模型中的所有可用参数，除了一个（学习率）之外怎么办？

使用 Fixed 条目（或任意数量的 Fixed 条目）传递超参数参数，并指定 tune_new_entries=True。

In [24]:
hypermodel = HyperXception(input_shape=(28, 28, 1), classes=10)

hp = HyperParameters()
hp.Fixed("learning_rate", value=1e-4)

tuner = RandomSearch(
    hypermodel,
    hyperparameters=hp,
    tune_new_entries=True,
    objective="val_accuracy",
    max_trials=3,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

tuner.search(
    x_train[:100], y_train[:100], epochs=1, validation_data=(x_val[:100], y_val[:100])
)

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.12999999523162842

Best val_accuracy So Far: 0.12999999523162842
Total elapsed time: 00h 00m 13s
INFO:tensorflow:Oracle triggered exit
time: 13.5 s (started: 2021-08-13 01:57:33 +08:00)


# 覆盖编译参数

如果您有一个想要更改现有优化器、损失或指标的超模型，您可以通过将这些参数传递给调谐器构造函数来实现：